# Simurgh project 


Simurgh is a set of open source tools and infrastructure that supports developing and evaluating algorithms (AI agents) for automated air traffic control (ATC).

For a full overview of the Simurch project (with install instructions), please read the project [README](https://github.com/alan-turing-institute/simurgh/blob/master/README.md).

This notebook runs through the full pipeline offered by Simurgh to train agents for ATC (using Python).

## Contents:

* [1. Getting started](#first-section)
* [2. Load sector and scenario](#second-section)
* [3. Example usage](#third-section)
* [4. Example OpenAI interface](#fourth-section)


## 1. Getting started <a class="anchor" id="first-section"></a>

### 1.1 Launch simulator (`BlueSky`) and the `BlueBird` interface layer

If you have not already started the BlueSky simulator and BlueBird, you can launch them using the command below:

In [ ]:
!docker-compose --file ../docker-compose.yml up --detach

### 1.2 Import `pydodo`

We will be using the `pydodo` package to communicate with BlueBird (and BlueSky). Assuming one has already installed it, we can import it here:

In [1]:
import pydodo

We can check the url PyDodo is using to communicate with BlueBird:

In [2]:
print(pydodo.bluebird_connect.get_bluebird_url())

http://localhost:5001/api/v2


If this is not correct (e.g., BlueBird is in fact being run on a different host), then we can change the url by using the `pydodo.bluebird_config` function:

In [3]:
help(pydodo.bluebird_config)

Help on function bluebird_config in module pydodo.bluebird_connect:

bluebird_config(host='localhost', port='5001', version='v2')
    Set BlueBird host, port and version parameters.
    Default values are taken from the config file.
    
    Parameters
    ----------
    host : str
        BlueBird host (e.g., 'localhost' or '0.0.0.0').
    port : int
        BlueBird port (e.g., 5001).
    version : str
        BlueBird version (e.g., 'v1' or 'v2')



We can check whether `pydodo` can succesfully communicate with BlueBird by querying aircraft positions:

In [4]:
pydodo.all_positions()

,aircraft_type,ground_speed,latitude,longitude,vertical_speed,heading,current_flight_level,requested_flight_level,cleared_flight_level


_Success!_

What do you mean success?! What we are seeing here is that `pydodo` is able to query positions of aircraft inside the simulator. The fact we do not see any is only becuase we have not defined a "Sector" and a "Scenario" yet.

## 2. Load sector and scenario <a class="anchor" id="second-section"></a>

### 2.1 Overview

To start, we need to upload a **sector** and a **scenario** (in that order). 

A sector defines the area the ATCO is controling. The scenario defines the episode: that is a sequence of aircraft whose route passes through the sector.

You can use the [Aviary](https://github.com/alan-turing-institute/aviary/tree/develop) package to create sector and scenario files (see the README for more information and file formats).

We can check whether a sector or a scenario has been defined already (see whether `sector_name` and `scenario_name` below are not `None`):

In [5]:
pydodo.simulation_info()

{'dt': 0.05,
 'mode': 'Agent',
 'scenario_name': None,
 'scenario_time': 0.0,
 'sector_name': None,
 'seed': None,
 'sim_type': 'BlueSky',
 'speed': 1.0,
 'state': 'INIT',
 'utc_datetime': '2020-04-10 00:00:00',
 'aircraft_ids': []}

Nothing has been loaded yet.

There is an example sector and scenario included with this example that we load below:

### 2.2 Load sector and scenario into `bluesky` using `bluebird`

In [10]:
pydodo.upload_sector('../dodo/PyDodo/tests/dodo-test-sector.geojson', 'test_sector')

True

In [11]:
pydodo.upload_scenario('../dodo/PyDodo/tests/dodo-test-scenario.json', 'test_scenario')

True

In [12]:
pydodo.simulation_info()

{'dt': 0.05,
 'mode': 'Agent',
 'scenario_name': 'test_scenario',
 'scenario_time': 4.65,
 'sector_name': 'test_sector',
 'seed': None,
 'sim_type': 'BlueSky',
 'speed': 1,
 'state': 'RUN',
 'utc_datetime': '2020-04-10 00:00:04',
 'aircraft_ids': ['VJ159', 'VJ405']}

## 3. Example usage <a class="anchor" id="third-section"></a>

See the Dodo [Specification document](https://github.com/alan-turing-institute/dodo/blob/master/Specification.md) for a detailed overview of the supported commands.

Below we show some of them.

### 3.1 Query aircraft positions, again

In [13]:
pydodo.all_positions()

,aircraft_type,cleared_flight_level,current_flight_level,ground_speed,heading,latitude,longitude,requested_flight_level,vertical_speed
VJ159,A346,400,39923.25,201,0,49.401176,-0.1275,400,-1500
VJ405,B77W,200,20000.00,141,180,53.567892,-0.1275,400,0


### 3.2 Check aircraft route

In [14]:
pydodo.list_route('VJ159')

{'next_waypoint': 'FIYRE',
 'route_name': 'ASCENSION',
 'route_waypoints': ['FIYRE', 'EARTH', 'WATER', 'AIR', 'SPIRT'],
 'aircraft_id': 'VJ159'}

### 3.3 Step through the simulation

By default, the simulation is in a paused state. To advance it, we need to call the `simulation_step` function:

In [15]:
pydodo.simulation_step()

True

This should result in the aircraft positions changing:

In [16]:
pydodo.all_positions()

,aircraft_type,cleared_flight_level,current_flight_level,ground_speed,heading,latitude,longitude,requested_flight_level,vertical_speed
VJ159,A346,400,36089.238845,215,0,49.705796,-0.1275,400,0
VJ405,B77W,200,20000.000000,187,180,53.320686,-0.1275,400,0


### 3.4 Change altitude

We can also instruct the aircraft to change their path (e.g., heading or altitude).

Below we instruct aircraft VJ405 to start climbing. This is followed by a step through the simulation:

In [17]:
pydodo.change_altitude("VJ405", flight_level = 300)

True

In [18]:
pydodo.simulation_step()

True

Checking aircraft positions now, we should see a change in the `current_flight_level` as the aircraft starts climbing:

In [21]:
pydodo.aircraft_position("VJ405")

,aircraft_type,cleared_flight_level,current_flight_level,ground_speed,heading,latitude,longitude,requested_flight_level,vertical_speed
VJ405,B77W,30000,20000.825,187,180,53.319003,-0.1275,400,165


## 4. Example OpenAI interface <a class="anchor" id="fourth-section"></a>

## TO DO!

In [ ]:
import gym
import birdhouse

env = gym.make("birdhouse-v1")

action = None

for _ in range(10):
    
    #==============================================
    # ENVIRONMENT STEP
    #==============================================
    obs, reward, done, info = env.step(action)
    
    #==============================================
    # AGENT - RULE BASED ACTION SELECTION
    #
    # 1. Choose aircraft with biggest absolute difference 
    #    between current and requested flight levels (FL).
    # 2. Send aircraft to requested FL 
    #.   (unless the difference is < 10 FL)
    #==============================================
    obs["FL_diff"] = (
        obs['requested_flight_level'] - obs['current_flight_level']/100
        ).abs()
    max_diff =  obs.loc[obs['FL_diff'].idxmax()]
    
    if max_diff['FL_diff'] >= 10:
        callsign = max_diff.name
        req_fl = max_diff['requested_flight_level']
        action = (callsign, req_fl)
    else:
        action = None